In [14]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

chromedriver.exe 경로 수정

In [ ]:
driver_path = 'C:/chromedriver_win32/chromedriver.exe' 

네이버에 언론사 설정하고 검색 시에 url_domain + 검색어 + news_domain 과 같은 구조  
그래서 아래 함수에서 news_domain을 각자 언론사에 맞춰서 수정해주시면 됩니다

In [17]:
def biz_khan(query,driver_path):
    domain = '경향신문'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
    url_domain = 'https://search.naver.com/search.naver?where=news&query='
    news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1032&nso=&is_sug_officeid=0' 
    url = url_domain + query + news_domain
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    # 기사 url을 list형태로 반환
    news_list = soup.select('#main_pack > section > div > div.group_news > ul')
    url_list = []
    news_list = news_list[0].find_all('li')
    
    for url in news_list:
        url_list.append(str(url.find_all('a')[0].get('data-url')))
    driver.close()
    
    data_list = []
    head_list = []
    for url in url_list:
        driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
        time.sleep(2)
        driver.get(url)
        time.sleep(2)
        # try문 안에서 각자 언론사에 맞게 수정
        try:
            soup = BeautifulSoup(driver.page_source,'html.parser')
            head_list.append(soup.find('h1',id='articleTtitle').text)
            tags = soup.select_one('#container > div.main_container > div.art_cont > div.art_body').find_all('p')
            data = [x.text for x in tags]
            data_list.append(' '.join(data))
        except:
            print('None_type')
        driver.close()
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

biz = biz_khan('삼성전자 슬로건',driver_path)

['http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201803220931001&code=920501', 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202006171100001&code=930301', 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201904232140005&code=930301', 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201807160600001&code=920101', 'http://www.khan.co.kr/kh_news/art_view.html?artid=201412090637171&code=920100', 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201708310110015&code=920100', 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201707301704001&code=920501', 'http://www.khan.co.kr/kh_news/art_view.html?artid=201401051107171&code=930301', 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201610302016005&code=920509', 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201703210802011&code=620109']
None_type


저장할 때

In [24]:
# biz.to_csv('경향신문.csv',encoding='CP949',index=None)

In [25]:
# test = pd.read_csv('경향신문.csv',encoding='CP949')

In [26]:
test

,언론사,제목,내용
0,경향신문,"삼성전자, 21일 밤 국내외 사업장서 ‘전등끄기’ 까닭은",삼성전자가 지난 21일 저녁 경기 수원 삼성 디지털시티와 44개 해외 판매·생산법인...
1,경향신문,삼성 가전 새 슬로건 ‘이제는 가전을 나답게’,ㆍ‘틀에 박힌 시장에 마침표’ 의미 담아…마케팅 전반에 활용 예정 삼성전자가 전 가...
2,경향신문,"철 없는 ‘4K TV’, 삼성에 효자 TV",게임 환경에서 QLED 4K TV(왼쪽)와 일반 TV의 화질을 비교한 모습. 삼성전...
3,경향신문,"[단독] 윤종원 청와대 경제수석 첫 인터뷰 “문 대통령·삼성 이재용 만남, 재벌 구...",·윤종원 신임 경제수석 임명 후 첫 언론 인터뷰 ·대통령이 삼성 이재용 부회장 만난...
4,경향신문,삼성그룹 새해 슬로건 ‘새로운 도전’···경영기조는 ‘내실’,삼성그룹이 새해 경영전략의 기조를 ‘내실 다지기’로 정했다. ‘새로운 도전’을 슬로...
5,경향신문,삼성 “당신 일상의 새 기준”…LG “더 나은 삶으로 혁신”,ㆍ국제가전전시회 내일 개막 삼성전자가 다음달 1일 독일 베를린에서 개막하는 ‘국제가...
6,경향신문,무풍 에어컨의 돌풍?···삼성 국내 에어컨 판매 100만대 돌파,삼성전자가 ‘무풍 에어컨’ 인기에 힘입어 올해 들어 가정용 에어컨 국내 판매 100...
7,경향신문,삼성전자 ‘스마트홈’ 사업 美 CES서 선보여,삼성전자는 TV·냉장고·세탁기 등 집안의 가전기기에다 스마트폰 같은 IT기기를 연결...
8,경향신문,[기업특집] “실패해도 괜찮아”…삼성의 도전은 계속된다,삼성전자 수원공장에는 C-스페이스와 아이디어 라운지라는 다소 이색적인 공간이 마련돼...
